# **1. Introduction**

Customer churn (AKA Customer attrition) occurs when a customer stops using a business's products or services. Customer retention is critical to the business as it is often more costly to acquire new customers compared to retaining existing ones. As such, predicting customer churn will help in early detection of customers who are at risks of leaving a service/product, allowing the business to take proactive measures in prioritising retention efforts with these customers. 

In [233]:
!pip install pydotplus
# !pip install six

In [234]:
#import libs
import pandas as pd
pd.set_option("display.max_columns", 101)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix ,classification_report,precision_score, recall_score ,f1_score, accuracy_score 

#decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image  
import pydotplus

In [235]:
#loading data
df = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# **2. Data Processing**
The overall data quality is quite good. There are no duplicates and outliers. There is only a handful of missing values for TotalCharges which was replaced with 0.

### 2A. Check data type, missing values and duplicates
As TotalCharges is a float and SeniorCitizen is a dummy variable, I have corrected the data type.

In [236]:
#check data type
df.dtypes

In [237]:
#convert data type
df['TotalCharges'] = df['TotalCharges'].str.replace(" ", "0", regex=False) #regex=False means input string is not a regex pattern
df['TotalCharges'] = df['TotalCharges'].astype('float64')
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')

In [238]:
#check for duplicate values - no duplicates
df['customerID'].duplicated().sum()

In [239]:
#drop irrelevant columns - customerID
df.index = df['customerID']
df = df.drop(columns='customerID')

In [240]:
#check for missing values
df.isnull().sum()

### 2B. Check for outliers
As seen from the summary stats table and boxplot below:
- tenure is positively skewed as mean is greater than median monthly charges
- MonthlyCharges is negatively skewed as mean is lower than median monthly charges
- TotalCharges is positively skewed as mean is greater than median total charges
- Based on the IQR formula, there are no outliers

In [241]:
#check outliers using mean and median
df.describe()

In [242]:
#check outliers using boxplot
numeric_col = ['tenure', 'MonthlyCharges', 'TotalCharges']
fig, ax = plt.subplots(1,3, figsize=(16, 4)) 

#enumerate adds a counter to iterable
for n, each_col in enumerate(numeric_col):
    ax[n].boxplot(df[each_col]) 
    ax[n].set_title(each_col)

# 3. Exploratory Data Analysis

In [243]:
numerical_col = []
cat_col = []
for each_feature in df.columns:
    if df[each_feature].dtypes == 'int64' or df[each_feature].dtypes == 'float64':
        numerical_col.append(each_feature)
    else:
        cat_col.append(each_feature)
        
print(numerical_col)
print(cat_col)

### 3A. Understand data distribution for each categorical variable
- In terms of demographic, this dataset is mainly made up customers who are young and has no dependents. 
- As for the firm's core services, 90% of the customers are subscribed to the firm's phone service. Out of these customers, majority of them own multiple lines. 77% of customers have subscribed to the firm's Internet service and out of these customers, 44% of customers are Fibre Optic users and 34% are DSL users.
- On average, across all the add-on service , 78% of customers do not subscribe or do not need add-on services such as Online Security, Online Backup, Device Protection, Tech Support, StreamingTV and StreamingMovies 
- Month-to-month contracts are more popular than 1-year and 2-year contracts
- This dataset has an unequal class distribution. 73% of the data represents existing customers while only 27% of the data represents churned customer. This could create an imbalanced classification issue where it will be more challeneging for the model to learn the characterisitics of the minority class (churned customers).

In [244]:
fig = plt.figure (figsize=(16, 18))
for y, each_feature in enumerate(cat_col[1:], start=1): #counter starts at 1, required by add_subplot
    x = fig.add_subplot(5,4,y) #5by4 plot, ax1 is subplot at position y
    x.pie(df.groupby(each_feature).size(), labels=df[each_feature].unique(), autopct=lambda p: '{:.0f}%'.format(p)) #autopct is the label for piechart, requires function
    x.set_title(each_feature) #set title for each subplot


### 3B. Understand relationship between numerical independent variable with dependent variable
- Churn customers have a shorter median tenure of 10 months, compared to non-churn customers who have a longer median tenure of 38 months
- Churn customers have a higher median monthly charge bill of \\$79.6 compared to non-churn customers who have a lower median monthly bill of \\$64.4.
- Yet, churn customers have a lower median total charge bill of \\$703.6 compared to non-churn customers who have a higher median total bill of \\$1079.5. This seems to contradict the findings from the monthly charge bill. A possible reason could be that churn customers have shorter tenure where they churn prematurely before the end of the fiscal period, resulting in lower total charges.

In [245]:
#numerical variables
fig, ax = plt.subplots(1,3, figsize=(16, 6)) 
for n, each_feature in enumerate(numeric_col):
    box_plot = sns.boxplot(ax=ax[n], y=each_feature, x = 'Churn', data=df)
    
    medians = round(df.groupby(df['Churn'])[each_feature].median(),1)
    vertical_offset = df[each_feature].median() * 0.05 # offset from median for display

    for xtick in box_plot.get_xticks():
        box_plot.text(xtick,medians[xtick] + vertical_offset,medians[xtick], 
                horizontalalignment='center',size='x-small',color='w',weight='semibold')

# 4. Classification Models

### 4A. Prepare data for analysis

In [246]:
#perform encoding for analysis
for each in cat_col:
    df[each] = df[each].astype('category')
    df[each] = df[each].cat.codes
    
df.head()

In [247]:
#split data into test and train
x = df.iloc[:,0:-1]
y = df.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=4)

### 4B. Correcting imbalanced class distribution
Since it was detected that the dataset has unequal class distribution, I have performed oversampling to duplicate samples from the minority class to create equally distributed classes in the training dataset. 

In [248]:
from imblearn.over_sampling import SMOTENC

sm = SMOTENC(categorical_features=[0,16], random_state = 2)
x_train_sm, y_train_sm = sm.fit_resample(x_train, y_train)

# what we had before
print("what we had before")
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# what we have now - an increase in training data, does not affect the test data
print("\nwhat we have now after oversampling")
print(x_train_sm.shape, y_train_sm.shape, x_test.shape, y_test.shape)

#class distribution
print("\nnew class distribution")
y_train_sm.value_counts()

### 4C. Model #1 - Logistic Regression
Logistic regression is a classification algorithm that predicts a binary outcome (churn vs non-churn). The dependent variable is categorical while the independent variables can be continuous or categorical. 

**Verifying Assumption #1: The independent variables are linearly related to the log odds**

Logistic Regression assumes that the continuous dependent variables are linearly related to the log odds of the dependent variable. To verify this assumption, I have plot the continuous dependent predictor - monthly charges and total charges to the log odds of the dependent variable. We would expect a flat top and bottom with a increasing/decreasing middle. Indeed, we have validated the assumption that the 2 continuous predictor variables are linearly related to the log odds of the dependent variable.

In [249]:
sns.regplot(x= 'MonthlyCharges', y= 'Churn', data= df, logistic= True).set_title("Monthly Charges Log Odds Linear Plot")

In [250]:
sns.regplot(x= 'TotalCharges', y= 'Churn', data= df, logistic= True).set_title("Total Charges Log Odds Linear Plot")

**Verifying Assumption #2: There is little to no multicollinearity between the independent variables**

Logistic Regression also assumes little to no multicollinearity. I have tested this by looking at the correlation between independent variables with Spearman and computed the variance inflation factor (VIF) score. In general, correlation > 0.7 and  VIF>10 indicates presence of multi-collinearity. As seen below, TotalCharges, Tenure, Phone Service and MonthlyCharges are at high risk of multi-collinearity and are removed from the dataset [1, 2]. 

In [251]:
sm_df = x.corr(method='spearman')
sm_df.style.applymap(lambda x: "background-color: red" if x>0.7 else "background-color: white")

In [252]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = x.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(x.values, i)
                          for i in range(len(x.columns))]
  
print(vif_data)

In [253]:
#drop columns with high multicollinearity
df = df.drop(columns=['MonthlyCharges','TotalCharges', 'PhoneService', 'tenure'])

In [254]:
output = x.columns[0]
for each in x.columns[1:]:
    output +=  " + " + each 
formula = 'Churn~ ' + output

**Model 1A - Logisitic Regression Model trained on original dataset**

In [255]:
#full dateset
model= smf.logit(formula=formula, data= x_train.join(y_train)).fit()
model.summary()

In [256]:
ypred = model.predict(x_test)
ypred
prediction = list(map(round, ypred))

# confusion matrix
cm = confusion_matrix(y_test, prediction)
labels=[0, 1]
df_cm = pd.DataFrame(cm, labels, labels)
ax = sns.heatmap(df_cm, annot=True, annot_kws={"size": 10}, square=True, cbar=False, fmt='g')
ax.set_ylim(0, 2) 
plt.xlabel("Predicted") 
plt.ylabel("Actual") 
ax.invert_yaxis() #optional
plt.show()
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction))

In [257]:
# Evaluating the predictive capability of the classification model

print(' precision score: ',precision_score(y_test, prediction),'\n')
print(' recall score: ',recall_score(y_test, prediction),'\n')
print(classification_report(y_test, prediction))

**Model 1B - Logisitic Regression Model trained on dataset corrected for imbalanced class**

In [258]:
#im dateset
model= smf.logit(formula=formula, data= x_train_sm.join(y_train_sm)).fit()
model.summary()

In [259]:
ypred = model.predict(x_test)
ypred
prediction = list(map(round, ypred))

# confusion matrix
cm = confusion_matrix(y_test, prediction)
labels=[0, 1]
df_cm = pd.DataFrame(cm, labels, labels)
ax = sns.heatmap(df_cm, annot=True, annot_kws={"size": 10}, square=True, cbar=False, fmt='g')
ax.set_ylim(0, 2) 
plt.xlabel("Predicted") 
plt.ylabel("Actual") 
ax.invert_yaxis() #optional
plt.show()
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction))

In [260]:
# Evaluating the predictive capability of the classification model

print(' precision score: ',precision_score(y_test, prediction),'\n')
print(' recall score: ',recall_score(y_test, prediction),'\n')
print(classification_report(y_test, prediction))

**Model Discussion**

- Although the first model (1A) produced a high accuracy score of 0.783, it has a low precision and recall scores at 0.594 and 0.524 respectively. The model performed well in terms of accuracy due to the accracy paradox - given that majority of the test data are of the majority class, a model that predicts the majority class for all the test data will have a high number of correct predictions. This results in a misleading accuracy score. 
- As such, I trained a second model (1B) with a balanced data set to evaluate if this would improve the recall and precision. Although the accuracy score decreased slightly to 0.746, the recall score has improved to 0.715. However, the precision score decreased to 0.508. This is the trade-off for correcting the dataset. As the model is now exposed to more data from the minority class (churn), more of its prediction now leans towards the minority class, resulting in higher false positives and thus, a lower precision. 
- Given both models, I would pick the second model (1B) as it has a higher recall. In this customer churn prediction case, false positives are less costly compared to false negatives. In a false positive case, the firm would proactively engage a non-churn customer as the model has wrongly predicted that the customer would churn. In a false negative case, the firm would neglect a churning customer as the model has wrongly predicted that the customer would not churn. The latter would cause greater financial loss to the firm as it is more expensive to acquire a new customer than to retain one. 

**Intepreting model output  [4]**
- StreamingMovies, PaperlessBilling and gender have no significant effect on the log odds of customers churning. (p-value>0.05)
- The remaining independent variables have a significant effect on the log odds of customer churning (p-value<0.05)
- Customers subscribed to phone service has a -1.43 decrease in log odds of churning compared to customers who are not subscribed to phone service
- Customers who have dependents has a -0.67 decrease in log odds of churning compared to customers who do not have dependents
- Senior citizen customers have a -0.61 decrease in log odds of churning compared to customers who are younger.


### 4D. Model #2 - Decision Tree
Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. 

In [261]:
clf = DecisionTreeClassifier(random_state=0)
clf = clf.fit(x_train_sm,y_train_sm)
y_pred = clf.predict(x_test)

In [262]:
# confusion matrix
cm = confusion_matrix(y_test, prediction)
labels=[0, 1]
df_cm = pd.DataFrame(cm, labels, labels)
ax = sns.heatmap(df_cm, annot=True, annot_kws={"size": 10}, square=True, cbar=False, fmt='g')
ax.set_ylim(0, 2) 
plt.xlabel("Predicted") 
plt.ylabel("Actual") 
ax.invert_yaxis() #optional
plt.show()
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, y_pred))

In [263]:
# Evaluating the predictive capability of the classification model

print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

**Fine-tuning Decision Tree [5]**

In general, the more complex our tree is, the more information it captures. However, this often leads to overfitting whereby the model will perform very well on the training data set but it will not be able to generalize well on the test data set. On the flip side, an overly simplified model will underfit as it is unable to learn meaningful patterns from the data and will perform badly on both train and test data set. 

There are several hyper-parameters that affect tree complexity:
- stopping criteria: max_depth, min_samples_split, and min_samples_leaf 
- pruning methods: min_weight_fraction_leaf and min_impurity_decrease

Here, I have decided to experiment with max_depth. As seen from the line chart below, a max_depth==3 gives the best score for both test and training data set. There are signs of overfitting for trees with max_depth above 7. The model's accuracy for train data set continue to increase while the model's accuracy for test data set has decreased.

In [264]:
values = [i for i in range(1, 21)]
train_scores, test_scores = list(), list()

# evaluate a decision tree for each depth
for i in values:
    clf = DecisionTreeClassifier(max_depth=i)
    clf = clf.fit(x_train, y_train)
    
    # evaluate on the train dataset
    train_ypred = clf.predict(x_train)
    train_acc = metrics.accuracy_score(y_train, train_ypred)
    train_scores.append(train_acc)
    
    # evaluate on the test dataset
    test_ypred = clf.predict(x_test)
    test_acc = metrics.accuracy_score(y_test, test_ypred)
    test_scores.append(test_acc)

# plot of train and test scores vs tree depth
fig, ax = plt.subplots(figsize=(12, 12))
ax.plot(values, train_scores, '-o', label='Train')
ax.plot(values, test_scores, '-o', label='Test')
ax.set_xticks([x for x in range(1,22)])
ax.legend()
plt.show()

In [265]:
clf = DecisionTreeClassifier(random_state=0, max_depth = 3)
clf = clf.fit(x_train_sm,y_train_sm)
y_pred = clf.predict(x_test)

In [266]:
# confusion matrix
cm = confusion_matrix(y_test, prediction)
labels=[0, 1]
df_cm = pd.DataFrame(cm, labels, labels)
ax = sns.heatmap(df_cm, annot=True, annot_kws={"size": 10}, square=True, cbar=False, fmt='g')
ax.set_ylim(0, 2) 
plt.xlabel("Predicted") 
plt.ylabel("Actual") 
ax.invert_yaxis() #optional
plt.show()
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, y_pred))

In [267]:
# Evaluating the predictive capability of the classification model

print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

In [268]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = x.columns ,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('DecisionTree.png')
Image(graph.create_png())

In [269]:
importances = clf.feature_importances_
weights = pd.Series(importances,
                 index=x.columns.values)
weights.sort_values()[-10:].plot(kind = 'barh')

### 4E. Model #3 - Random Forest
A random forest is an ensemble method that uses averaging from various classifiers to improve the predictive accuracy and control over-fitting

In [270]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=350, max_depth=3)
clf.fit(x_train_sm,y_train_sm)
y_pred=clf.predict(x_test)

In [271]:
# confusion matrix
cm = confusion_matrix(y_test, prediction)
labels=[0, 1]
df_cm = pd.DataFrame(cm, labels, labels)
ax = sns.heatmap(df_cm, annot=True, annot_kws={"size": 10}, square=True, cbar=False, fmt='g')
ax.set_ylim(0, 2) 
plt.xlabel("Predicted") 
plt.ylabel("Actual") 
ax.invert_yaxis() #optional
plt.show()
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, y_pred))

# 5. Comparing model performance
- Overall, the logistic regression model performs better compared to decision tree model. The logistic regression model gave a better accuracy score of 0.746 and macro-F1 score of 0.79. In comparison, the decision tree model produced a lower accuracy score of 0.689 and macro-F1 score of 0.66. 

#### Key recommedations:
- The InternetService type has a 0.2851 increase in log odds ratio of customer churning. I would recommend the firm to conduct a survey on satisfaction with Internet Service to understand if there might be any underlying issues with the InternetService that may cause customers to churn.
- As younger customers with no dependents and no partners are more likely to churn, I would recommend the firm to create bundle family packages with their older family members / friends. 
- For every one unit increase in monthly charges, the logg odds of customer churning increase by 0.029. Based on the EDA findings, churn customers have a higher median monthly charge compared to non-churn customers. As such, I would recommend the firm to investigate the underlying the possible causes of higher median monthly charge and take steps to reduce the cost to prevent losing these customers to other competitors. 

# References:
* [1] https://blog.clairvoyantsoft.com/correlation-and-collinearity-how-they-can-make-or-break-a-model-9135fbe6936a#:~:text=Multicollinearity%20is%20a%20situation%20where,indicates%20the%20presence%20of%20multicollinearity.
* [2] https://quantifyinghealth.com/vif-threshold/
* [3] https://www.geeksforgeeks.org/logistic-regression-using-statsmodels/
* [4] https://pythonfordatascienceorg.wordpress.com/logistic-regression-python/
* [5] https://towardsdatascience.com/how-to-tune-a-decision-tree-f03721801680